# Test data loader

In [ ]:
# make data splits

import os
import random
from glob import glob
import shutil

data_splits = ['train', 'val', 'test']
data_root = "/home/lry/data/AIGP/demo_v2/Q2/imgs"
all_items = glob(os.path.join(data_root, '*.npz'))

print("Total items: ", len(all_items))

random.shuffle(all_items)
num_val = 100
num_test = 100
num_train = len(all_items) - num_val - num_test

print(num_train, num_val, num_test)

for split in data_splits:
    split_dir = os.path.join(data_root, split)
    if not os.path.exists(split_dir):
        os.makedirs(split_dir)
    if split == 'train':
        items = all_items[:num_train]
    elif split == 'val':
        items = all_items[num_train:num_train+num_val]
    elif split == 'test':
        items = all_items[num_train+num_val:]
    print(split, len(items), items[0])
    for item in items:
        shutil.move(item, os.path.join(split_dir, os.path.basename(item)))

In [ ]:
import os
import random
from glob import glob

import numpy as np

import torch
import torch.nn.functional as F
from torchvision import transforms
from diffusers.models.embeddings import SinusoidalPositionalEmbedding

from matplotlib import pyplot as plt
from matplotlib.colors import to_rgba

_CMAP = {
    "帽": {"alias": "hat", "color": "#F7815D"},
    "领": {"alias": "collar", "color": "#F9D26D"},
    "肩": {"alias": "shoulder", "color": "#F23434"},
    "袖片": {"alias": "sleeve", "color": "#C4DBBE"},
    "袖口": {"alias": "cuff", "color": "#F0EDA8"},
    "衣身前中": {"alias": "body front", "color": "#8CA740"},
    "衣身后中": {"alias": "body back", "color": "#4087A7"},
    "衣身侧": {"alias": "body side", "color": "#DF7D7E"},
    
    "底摆": {"alias": "hem", "color": "#DACBBD"},
    "腰头": {"alias": "belt", "color": "#DABDD1"},
    "裙前中": {"alias": "skirt front", "color": "#46B974"},
    "裙后中": {"alias": "skirt back", "color": "#6B68F5"},
    "裙侧": {"alias": "skirt side", "color": "#D37F50"},

    "橡筋": {"alias":"ruffles", "color": "#A8D4D2"},
    "木耳边": {"alias":"ruffles", "color": "#A8D4D2"},
    "袖笼拼条": {"alias":"ruffles", "color": "#A8D4D2"},
    "荷叶边": {"alias":"ruffles", "color": "#A8D4D2"},
    "绑带": {"alias":"ruffles", "color": "#A8D4D2"}
}

_PANEL_CLS = [
    '帽','领','肩','袖片','袖口','衣身前中','衣身后中','衣身侧','底摆','腰头','裙前中','裙后中','裙侧','橡筋','木耳边','袖笼拼条','荷叶边','绑带']

_PANEL_COLORS = np.array(
    [(0., 0., 0., 0.)] + [to_rgba(_CMAP[_PANEL_CLS[idx]]['color']) for idx in range(len(_PANEL_CLS))]
)

_RESO = 1024


def _positional_encoding(input, num_freq, min_freq, max_freq, include_input=False):
    
    if not torch.is_tensor(input): input = torch.tensor(input)

    scaled = 2 * torch.pi * input  # scale to [0, 2pi]
    
    freqs = 2 ** torch.linspace(min_freq, max_freq, num_freq, device=input.device)
        
    print("*** freqs: ", freqs.shape, scaled.shape)
        
    scaled = scaled[..., None] * freqs  # [..., "input_dim", "num_scales"]
    scaled = scaled.view(*scaled.shape[:-2], -1)  # [..., "input_dim" * "num_scales"]
        
    pos_feat = torch.sin(torch.cat([scaled, scaled + torch.pi / 2.0], dim=-1))

    if include_input: pos_feat = torch.cat([pos_feat, input], dim=-1)
    
    return pos_feat


def _vis_tensor(geo_arr, seg_arr, uv_arr, tag="ORIGINAL"):
    
    print(f'>>>>>>>>> {tag} <<<<<<<<<')
    print('*** geo_img: ', geo_arr.shape, geo_arr.min(), geo_arr.max())
    print('*** seg_img: ', seg_arr.shape, np.unique(seg_arr))
    print('*** uv_img: ', uv_arr.shape, uv_arr.min(), uv_arr.max())
    print('||||||||||||||||||||||||||||')
    
    if torch.is_tensor(geo_arr): geo_arr = geo_arr.numpy().transpose(1, 2, 0)
    if torch.is_tensor(seg_arr): seg_arr = seg_arr.numpy()
    if torch.is_tensor(uv_arr): uv_arr = uv_arr.numpy().transpose(1, 2, 0)

    _, axs = plt.subplots(1, 3, figsize=(12, 4))
    mask_arr = seg_arr > 0
    geo_img = np.clip(np.rint((geo_arr + 1.0) * 0.5 * 255) * mask_arr[..., None], 0, 255).astype(np.uint8)
    seg_img = np.clip(np.rint(_PANEL_COLORS[seg_arr, :] * 255), 0, 255).astype(np.uint8)
    uv_img = np.clip(np.rint((uv_arr + 1.0) * 0.5 * 255) * mask_arr[..., None], 0, 255).astype(np.uint8)
    
    axs[0].axis('off')
    axs[0].set_title('Geometry Image')
    axs[0].imshow(geo_img)
    
    axs[1].axis('off')
    axs[1].set_title('Semantic Image')
    axs[1].imshow(seg_img)
    
    axs[2].axis('off')
    axs[2].set_title('UV Image')
    if uv_img.shape[-1] == 2: uv_img = np.concatenate([uv_img, np.zeros_like(uv_img[..., :1])], axis=2)
    axs[2].imshow(uv_img)
    
    plt.show()
    
# data_root = "E:\\lry\\data\\AIGP\\demo_v2\\Q2\\imgs_test_1"
data_root = "/home/lry/data/AIGP/demo_v2/Q2/imgs"
data_item = random.choice(glob(os.path.join(data_root, '**', '*.npz')))
print(data_item)

data = np.load(data_item)
geo_arr = data['geo']
seg_arr = data['seg']
uv_arr = data['uv']

_vis_tensor(geo_arr, seg_arr, uv_arr, tag="ORIGINAL")

# ############ Test positional encoding ############
# H, W = geo_arr.shape[0], geo_arr.shape[1]
# coord_arr = np.stack(np.meshgrid(
#     np.linspace(0., 1.0, W),
#     np.linspace(0., 1.0, H)
# ), axis=-1) * ((seg_arr[..., None] > 0).astype(np.float32))
# # print('*** coord_arr: ', coord_arr.shape, coord_arr[0, 0, :], coord_arr[-1, 0, :], coord_arr[0, -1, :], coord_arr[-1, -1, :])

# plt.imshow(np.concatenate([coord_arr, np.zeros_like(coord_arr[..., :1])], axis=2))
# plt.show()

# encoded_coord_arr = _positional_encoding(coord_arr, 4, 2, 8, include_input=False)
# # print('*** encoded_coord_arr: ', encoded_coord_arr.shape, encoded_coord_arr.min(), encoded_coord_arr.max())

# plt.imshow((encoded_coord_arr[..., 6:9] + 1.0) * 0.5)
# plt.show()
# ####################################################

data_tensor = torch.from_numpy(
    np.concatenate([geo_arr, seg_arr[..., None], uv_arr], axis=2).transpose(2, 0, 1)).to(torch.float32)

data_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(
            _RESO, interpolation=transforms.InterpolationMode.NEAREST_EXACT,
            scale=(0.25, 1.0), ratio=(0.5, 2.0)
        ),
        transforms.RandomHorizontalFlip()
    ]
)

transformed_tensor = data_transform(data_tensor)
geo_arr = transformed_tensor[:3]
seg_arr = transformed_tensor[3].long()
uv_arr = transformed_tensor[4:]

_vis_tensor(geo_arr, seg_arr, uv_arr, tag="TRANSFORMED")

# Test Inference

In [12]:
import os
import sys

import numpy as np

import torch
import torchvision.transforms.functional as F
from torchvision.utils import make_grid

from matplotlib import pyplot as plt
from matplotlib.colors import to_rgb

from tqdm import tqdm
from glob import glob

import plotly.offline as py
import plotly.graph_objects as go


PROJ_ROOT = '/home/lry/code/image2pattern/thirdparty/automodeling'
sys.path.insert(0, PROJ_ROOT)

from datasets import SXDImageDataset
from network import SXDUNet2DModel

_CMAP = {
    "帽": {"alias": "hat", "color": "#F7815D"},
    "领": {"alias": "collar", "color": "#F9D26D"},
    "肩": {"alias": "shoulder", "color": "#F23434"},
    "袖片": {"alias": "sleeve", "color": "#C4DBBE"},
    "袖口": {"alias": "cuff", "color": "#F0EDA8"},
    "衣身前中": {"alias": "body front", "color": "#8CA740"},
    "衣身后中": {"alias": "body back", "color": "#4087A7"},
    "衣身侧": {"alias": "body side", "color": "#DF7D7E"},
    
    "底摆": {"alias": "hem", "color": "#DACBBD"},
    "腰头": {"alias": "belt", "color": "#DABDD1"},
    "裙前中": {"alias": "skirt front", "color": "#46B974"},
    "裙后中": {"alias": "skirt back", "color": "#6B68F5"},
    "裙侧": {"alias": "skirt side", "color": "#D37F50"},

    "橡筋": {"alias":"ruffles", "color": "#A8D4D2"},
    "木耳边": {"alias":"ruffles", "color": "#A8D4D2"},
    "袖笼拼条": {"alias":"ruffles", "color": "#A8D4D2"},
    "荷叶边": {"alias":"ruffles", "color": "#A8D4D2"},
    "绑带": {"alias":"ruffles", "color": "#A8D4D2"}
}

_PANEL_CLS = [
    '帽','领','肩','袖片','袖口','衣身前中','衣身后中','衣身侧','底摆','腰头','裙前中','裙后中','裙侧','橡筋','木耳边','袖笼拼条','荷叶边','绑带']

_PANEL_COLORS = np.array(
    [(0., 0., 0.)] + [to_rgb(_CMAP[_PANEL_CLS[idx]]['color']) for idx in range(len(_PANEL_CLS))]
)

_AVATAR_BBOX = np.array([
    [-449.00006104,  191.10876465, -178.34872437],      # min
    [ 447.45980835, 1831.29016113,  174.13575745]       # max
])

_OFFSET = _AVATAR_BBOX[0] + (_AVATAR_BBOX[1] - _AVATAR_BBOX[0]) / 2
_SCALE = _AVATAR_BBOX[1] - _AVATAR_BBOX[0]

plt.rcParams["savefig.bbox"] = 'tight'
def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])


expr_name = 'cf_local_256_20240808'
ckpt_step = 14000

use_cf, uv_type, reso, _ = expr_name.split('_')

ckpt_dir = os.path.join(PROJ_ROOT, 'ckpts', expr_name, 'checkpoint-%d'%(ckpt_step))
data_root = "/data/lry/StyleXD/imgs"
        
dataset = SXDImageDataset(data_root, reso=int(reso), split='test', uv_type=uv_type)
model = SXDUNet2DModel(
        sample_size=int(reso),
        in_channels=16, 
        num_class_embeds=dataset.num_classes if use_cf == 'cf' else -1, 
        out_channels=3, 
        layers_per_block=1,
        block_out_channels=(32, 64, 128, 128),
        down_block_types=(
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
            "DownBlock2D",
        ),
        up_block_types=(
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
            "UpBlock2D",
        )
    )

load_model = SXDUNet2DModel.from_pretrained(ckpt_dir, subfolder="unet")
model.register_to_config(**load_model.config)
model.load_state_dict(load_model.state_dict())
model.to('cuda')

output_root = '/home/lry/code/image2pattern/thirdparty/automodeling/out/cf_local_256_20240808'
vis_output_root = os.path.join(output_root, 'vis')
os.makedirs(vis_output_root, exist_ok=True)

for data_fp in tqdm(sorted(glob(os.path.join(output_root, '*.npz')))):

    # geo_arr, seg_arr, uv_arr = dataset[0]        
    # seg_mask = (seg_arr > 0)
    # # print(geo_arr.shape, seg_arr.shape, uv_arr.shape, geo_arr.min(), geo_arr.max())
    # # show(make_grid((geo_arr.detach() + 1.0) * 0.5)) 

    # with torch.no_grad():
    #     pred = model(uv_arr[None].to('cuda'), seg_arr[None].to('cuda')).sample[0]
    # # print(pred.shape, pred.max(), pred.min())
    # # show(make_grid((pred.detach() + 1.0) * 0.5))

    data = np.load(data_fp)
    geo_arr = data['pred']
    mask_arr = data['seg'] > 0
    # show(make_grid((torch.from_numpy((geo_arr + 1.0) * 0.5 * mask_arr[None]))))

    traces = []
    mask_arr = data['seg'] > 0
    error_map = np.linalg.norm(data['geo'] - data['pred'], axis=0)

    for key in ['geo', 'pred']:
        geo_arr = data[key]
        valid_pts = geo_arr.reshape(3, -1)[:, mask_arr.reshape(-1)]
        valid_pts = valid_pts * (_SCALE[:, None] * 0.5) + _OFFSET[:, None]
        pts_cls = seg_arr.reshape(-1)[mask_arr.reshape(-1)]
        
        if key == 'geo':
            traces.append(
                go.Scatter3d(
                    x=valid_pts[0, :],
                    y=valid_pts[1, :],
                    z=valid_pts[2, :],
                    mode='markers',
                    marker=dict(
                        color='#D3D3D3',  # Light grey for ground truth
                        size=1
                    )
                )
            )
        else:
            error_values = error_map.reshape(-1)[mask_arr.reshape(-1)]  # Apply the error map to valid points
            print(error_values.min(), error_values.max())
            traces.append(
                go.Scatter3d(
                    x=valid_pts[0, :],
                    y=valid_pts[1, :],
                    z=valid_pts[2, :],
                    mode='markers',
                    marker=dict(
                        color=error_values,  # Color by error values
                        colorscale='Rainbow',  # Choose a colorscale, e.g., Viridis
                        size=2,
                        colorbar=dict(title="Reconstruction Error"),
                        showscale=False,
                        cmin=0.0,  # Set the minimum value of the color scale
                        cmax=1.0 
                    )
                )
            )

    fig = go.Figure(data=traces,
        layout=go.Layout(
            width=1024,
            height=1024,
            margin=dict(l=1, r=1, b=1, t=1),
            scene=dict(aspectmode='data',xaxis = dict(visible=False),yaxis = dict(visible=False),zaxis = dict(visible=False)),
            showlegend=False,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
    )

    camera = dict(
        up=dict(x=0, y=1, z=0),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=0, y=0, z=2.0)
    )

    fig.update_layout(scene_camera=camera, title="Reconstructed point cloud.")
    # fig.show()
    # py.iplot(fig, filename='3d-scatter')
    fig.write_image(os.path.join(vis_output_root, os.path.basename(data_fp).replace('npz', 'png')))

Load test split from /data/lry/StyleXD/imgs, 115 items.
*** data transform:  Resize(size=[256, 256], interpolation=nearest-exact, max_size=None, antialias=True)


  0%|          | 0/115 [00:00<?, ?it/s]

0.0026174102 0.62751925


  1%|          | 1/115 [00:02<04:12,  2.22s/it]

0.002329227 0.4756424


...

 98%|█████████▊| 113/115 [04:57<00:05,  2.79s/it]

0.002627177 0.28940052


 99%|█████████▉| 114/115 [05:00<00:02,  2.89s/it]

0.0037446686 1.4044725


100%|██████████| 115/115 [05:03<00:00,  2.64s/it]
